<a href="https://colab.research.google.com/github/elhamod/IS883/blob/main/Week10/IS883_2024_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IS883 Week10: Tools and Agents

1. Use Google Colab for this assignment.

2. **You are NOT allowed to use external or embedded Gen AI for this assignment (except where specifically instructed). However, you may use Google search and other online resources. As per the syllabus, you are required to cite your usage. You are also responsible for understanding the solution and defending it when asked in class.**

3. For each question, fill in the answer in the cell(s) right below it. The answer could be code or text. You can add as many cells as you need for clarity.

4. **Your submission on Blackboard should be the downloaded notebook (i.e., ipynb file). It should be prepopulated with your solution (i.e., the TA and/or instructor need not rerun the notebook to inspect the output). The code, when executed by the TA and/or instructor, should run with no runtime errors.**

#1. In-Class Work

In [ ]:
!pip install langchain langchain_community langchain_openai

In [ ]:
### Get the OpenAI API key
from google.colab import userdata
openai_api_key = userdata.get('MyOpenAIKey')

## 1.1 Tools


###1.1.1 Using a Calculator for Accurate Computations.

Let's try to get an answer to the following finantial problem:



```
In your bank account, you have $110,345.45. Because you left them in a saving account with a high annual interest rate of %6.46, how much money will you have in a year?
```

The correct answer is $110,345.45 * 1.0646 = \$117473.76607


In [ ]:
question = "In your bank account, you have $110,345.45. Because you left them in a saving account with a high annual interest rate of %6.46, how much money will you have in a year?"

In [ ]:
from langchain_openai import ChatOpenAI
from IPython.display import Markdown

### Create the chat agent
chat = ChatOpenAI(openai_api_key=openai_api_key, model="gpt-4o")

### Get the answer
display(Markdown(chat.invoke(question).content))

To calculate the amount of money you will have in a year with an annual interest rate of 6.46%, you can use the formula for compound interest. Assuming the interest is compounded annually, the formula is:

\[ A = P(1 + r)^n \]

Where:
- \( A \) is the amount of money accumulated after n years, including interest.
- \( P \) is the principal amount (initial investment).
- \( r \) is the annual interest rate (decimal).
- \( n \) is the number of years the money is invested for.

Given:
- \( P = 110,345.45 \)
- \( r = 6.46\% = 0.0646 \)
- \( n = 1 \)

Plug these values into the formula:

\[ A = 110,345.45 \times (1 + 0.0646)^1 \]

\[ A = 110,345.45 \times 1.0646 \]

\[ A \approx 117,475.78 \]

So, you will have approximately $117,475.78 in your account after one year.

**Question:** Is the answer correct? What do you think is the cause for what you observe?

ChatGPT only predicts tokens based on the text it has seen during training. It does not do any true calculations. As such, if this specific example is unlikely to have appeared in  training dataset, then the result is unlikely to be correct.

In order to upgrade the LLM's capabilities, we will use [`LLMMathChain`](https://medium.com/data-science-in-your-pocket/mathematics-using-llms-using-langchains-ca23bbd1a38b), which is a chain that uses a calculator tool, into our framework.



In [ ]:
from langchain import LLMMathChain
from langchain_openai import OpenAI

### You can turn on debugging using this code. You will be able to see the intermediate requests and responses.
import langchain
langchain.debug = False

### Create the LLM
llm = OpenAI(openai_api_key=openai_api_key)

### Wrap the LLM with a chain that has access to the math tool.
llm_math = LLMMathChain.from_llm(llm) #, verbose=True

### Run the chain
llm_math.invoke(question)

{'question': 'In your bank account, you have $110,345.45. Because you left them in a saving account with a high annual interest rate of %6.46, how much money will you have in a year?',
 'answer': 'Answer: 117473.76607'}

**Question:** What are the steps `LangChain` took to obtain the answer this time?

*Hint: You may need some diagnostics...*

- The LLM adds a prompt as a prefix to your question:


```
    "Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.
    \n
    \nQuestion: ${Question with math problem.}
    \n```text
    \n${single line mathematical expression that solves the problem}
    \n```
    \n...numexpr.evaluate(text)...
    \n
    ```output
    \n${Output of running the code}
    \n```
    \nAnswer: ${Answer}
    \n
    \nBegin.
    \n
    \nQuestion: What is 37593 * 67?
    \n```text
    \n37593 * 67
    \n```
    \n...numexpr.evaluate(\"37593 * 67\")...
    \n```output
    \n2518731
    \n```
    \nAnswer: 2518731
    \n
    \nQuestion: 37593^(1/5)
    \n```text\n37593**(1/5)
    \n```
    \n...numexpr.evaluate(\"37593**(1/5)\")...
    \n```
    \n8.222831614237718
    \n```
    \nAnswer: 8.222831614237718
    \n
    \nQuestion: In your bank account, you have $110,345.45. Because you left them in a saving account with a high annual interest rate of %6.46, how much money will you have in a year?"
```

- It obtains the expression:



```
text
\n110345.45 * 1.0646
\n```
\n...numexpr.evaluate(\"110345.45 * 1.0646\")...
\n
```

- It *executes* the expression as code (not through the LLM itself), and returns the answer.

###1.1.2 Tool Binding. Use-Case: Websearching Using Google Serper.



Now, let's say we want a smart personal assistant bot that can browse the internet and be helpful.

We will use [Google Serper](https://python.langchain.com/docs/integrations/tools/google_serper/), a *free* search tool.

`LangChain` has some built-in [tools](https://python.langchain.com/docs/integrations/tools/) and [toolkits](https://python.langchain.com/v0.1/docs/integrations/toolkits/) that provide access to some useful functionality and APIs.

The following [link](https://python.langchain.com/docs/how_to/tools_chain/) shows how to use tools in chains.

In [ ]:
question = "Based on the weather today in Miami, do you think it is a good idea to go on a picnic?"
# question = "I am at Questrom and hungry. give me 3 good noodle places near me?"
# question = "What phone number should I call to talk to Mohannad Elhamod at Questrom?"
# question = "What is 1+1"

In [ ]:
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.tools import Tool
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent
import os


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"), # To be used by the agent for intermediate operations.
    ]
)

chat = ChatOpenAI(openai_api_key=openai_api_key, model="gpt-4o-mini")

# Setting up the Serper tool
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API')
search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name="GoogleSerper",
        func=search.run,
        description="Useful for when you need to look up some information on the internet.",
    )
]

# Defining the agent
agent = create_tool_calling_agent(chat, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) #, verbose=True

print("Vanilla LLM answer:", chat(question).content)

# Run the agent
print("*****")
print("Agent answer:", agent_executor.invoke({"input": question})["output"])



<ipython-input-7-002150cb95e7>:33: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print("Vanilla LLM answer:", chat(question).content)


Vanilla LLM answer: I don't have access to real-time weather data, so I can't provide specific advice based on today's conditions in Miami. However, when considering a picnic, you should check the current temperature, the chance of rain, humidity levels, and wind conditions. A pleasant day with mild temperatures and low humidity is ideal for a picnic. If the forecast indicates rain or extreme heat, it might be better to postpone your plans. Always remember to bring necessary items like sunscreen, water, and insect repellent!
*****


> Entering new AgentExecutor chain...

Invoking: `GoogleSerper` with `Miami weather today`


77°FThe weather in Miami today is 77°F. Generally, this temperature can be pleasant for a picnic, but other factors such as humidity, wind, and the likelihood of rain can also play a significant role. If the weather is sunny and not too humid, it could be a great day for a picnic. Would you like to know more about the specific weather conditions such as humidity or 

###1.1.3 Custom Tool Binding. Use-Case: What's the time?

Let's say we want out LLM to be aware of today's date. We want to implement our own date-checking function. This is done using `@tool` decorator.

A tool expects some input parameters, if any, and returns some output. Here is some overview of [how tool calling is done](https://python.langchain.com/docs/concepts/tool_calling/#:~:text=A%20key%20principle%20of%20tool,result%20%3D%20llm_with_tools.). Here is a resource on how to build [custom tools](https://python.langchain.com/v0.1/docs/use_cases/tool_use/quickstart/).

Reference: [`create_tool_calling_agent`](https://api.python.langchain.com/en/latest/agents/langchain.agents.tool_calling_agent.base.create_tool_calling_agent.html)

Reference: [custom tools](https://python.langchain.com/docs/how_to/custom_tools/)

In [ ]:
question = "What's the date today?"
# question = "Give a singer who is alive whose birthdate is next month"
# question = "Give a celebrity whose birthdate is in 6 weeks"
# question = "When was Michael Jackson born?"
# question = "What day of the week is in 3 days?"
# question = "What year is it?"
# question = "I ate a date. Will I be OK?"
# question = "What would be a good venue for a romantic date in Boston?"

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

import langchain
langchain.debug = False

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Defining the tools
from datetime import date, timedelta, datetime
@tool
def datetoday() -> str:
    """Returns today's date, use this for any \
    questions that need today's date to be answered. \
    This tool takes no argumetns but returns a string with today's date.""" #This is the desciption the agent uses to determine whether to use the time tool.
    return "Today is " + str(date.today())

tools = [datetoday]


# Defining the agent
agent = create_tool_calling_agent(chat, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) #


print("Vanilla LLM answer:", chat([HumanMessage(content=question)]).content)

# # Run the agent
print("*****")
print("Agent answer:", agent_executor.invoke({"input": question})["output"])

Vanilla LLM answer: Today's date is October 3, 2023.
*****


> Entering new AgentExecutor chain...

Invoking: `datetoday` with `{}`


Today is 2024-11-18Today is November 18, 2024.

> Finished chain.
Agent answer: Today is November 18, 2024.


##1.2 Chain of Thought (CoT)

As we have seen in class before, breaking down a complex task into steps is useful as it generates intermediate steps, yielding a better and more accurate final answer.

Here, we will take this idea further and apply *Chain of Thought (CoT)*, a prompt engineering technique that asks the model to "reason" through its answer by providing intermediate outputs.

You may read this [reference](https://www.promptingguide.ai/techniques/cot) for more information.

###1.2.1 Do Objects Float. Yes or No?



> Would an avocado float in medical alcohol?

The answer lies in object densities:
* ethanol (grain alcohol):	0.810
*  [Avocado density](https://www.sciencedirect.com/science/article/pii/S0260877422002734): 1.02



Let's try it out. We will use the `PromptTemplate` format to promote flexibilty (i.e., we may be interested in trying different liquids or objects).



In [ ]:
obj = "avocado"
liquid = "medical alcohol"

In [ ]:
from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback

instruction = "Only provide the answer as a single yes or no without an explanation: \n"
riddle = """
Would a {obj} float in {liquid}?
"""

chat = ChatOpenAI(openai_api_key=openai_api_key, model="gpt-4o-mini")

riddle_prompt_template =PromptTemplate(input_variables=["obj", "liquid"], template=instruction + riddle)
simple_chain = LLMChain(llm=chat, prompt=riddle_prompt_template)

# print("Answer with no reasoning")
avg_cost = 0
for i in range(10):
  with get_openai_callback() as cb:
    print("Answer: ", simple_chain.invoke({"obj":obj, "liquid":liquid})["text"], "Cost: $", cb.total_cost)
    avg_cost += cb.total_cost

avg_cost = avg_cost/10
print("Average cost: $", avg_cost)

<ipython-input-11-1568a1882fc3>:15: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  simple_chain = LLMChain(llm=chat, prompt=riddle_prompt_template)


Answer:  No. Cost: $ 5.7e-06
Answer:  No. Cost: $ 5.7e-06
Answer:  No. Cost: $ 5.7e-06
Answer:  No. Cost: $ 5.7e-06
Answer:  Yes. Cost: $ 5.7e-06
Answer:  Yes. Cost: $ 5.7e-06
Answer:  Yes. Cost: $ 5.7e-06
Answer:  Yes. Cost: $ 5.7e-06
Answer:  No. Cost: $ 5.7e-06
Answer:  No. Cost: $ 5.7e-06
Average cost: $ 5.7000000000000005e-06


**Question:**

- What happens if I remove the instruction? Is that a problem?

###1.2.2 Do Objects Float? Can Gen AI figure it out?

Let's let `GPT-4o` do what it needs to do to get the answer without restraints, and then extract the answer and return it. We can do this using `LLMChain`s

In [ ]:
from langchain.chains import SimpleSequentialChain

chat = ChatOpenAI(openai_api_key=openai_api_key, model="gpt-4o-mini")

# riddle prompt where object and liquid are substituted.
riddle_prompt =PromptTemplate(input_variables=["obj", "liquid"], template=riddle).format_prompt(obj=obj, liquid=liquid)

# The first chain solves the riddle.
solver_prompt = PromptTemplate(
    input_variables=["riddle"], template="Solve the following riddle: {riddle}"
)
solver_chain = LLMChain(llm=chat, prompt=solver_prompt)

# The second chain extracts the final answer.
extractor_prompt = PromptTemplate(
    input_variables=["input"], template="Given the input, extract the answer as a yes or no with no explanation or additional text. Input: {input}"
)
extractor_chain = LLMChain(llm=chat, prompt=extractor_prompt)

# The overall chain
ss_chain = SimpleSequentialChain(chains=[solver_chain, extractor_chain]) # ,verbose=True

print("Answer with optional reasoning")
avg_cost = 0
for i in range(10):
  with get_openai_callback() as cb:
    result = ss_chain.invoke(riddle_prompt)
    print("Answer: ", result["output"], "Cost: $", cb.total_cost)
    avg_cost += cb.total_cost

avg_cost = avg_cost/10
print("Average cost: $", avg_cost)

Answer with optional reasoning
Answer:  No Cost: $ 9.944999999999999e-05
Answer:  No Cost: $ 0.0001362
Answer:  Yes Cost: $ 0.00012119999999999999
Answer:  No Cost: $ 0.0001317
Answer:  No Cost: $ 8.895e-05
Answer:  Yes Cost: $ 0.00014144999999999997
Answer:  Yes Cost: $ 0.00011295
Answer:  Yes Cost: $ 0.0001467
Answer:  No Cost: $ 0.00010995
Answer:  No Cost: $ 0.00015644999999999998
Average cost: $ 0.0001245


**Question:**

What happens if we upgrade the model?

###1.2.3 Do Objects Float? Can we nudge Gen AI to figure it out?

So far, we had to either increase cost or accept low performance. Can we do better by telling the model how to solve the problem?

In [ ]:
instructions = """
- Follow these steps:
1) what is the density of {obj}?
1) what is the density of {liquid}?
2) is density of  {obj} is less than a  {liquid} then yes, otherwise no.
"""

chat = ChatOpenAI(openai_api_key=openai_api_key, model="gpt-4o-mini")

# chain for substituting
riddle_prompt =PromptTemplate(input_variables=["obj", "liquid"], template=riddle + instructions).format_prompt(obj=obj, liquid=liquid)

# The first chain solves the riddle.
solver_prompt = PromptTemplate(
    input_variables=["riddle"], template="Solve the following riddle: {riddle}. "
)
solver_chain = LLMChain(llm=chat, prompt=solver_prompt)

# The second chain extracts the final answer.
extractor_prompt = PromptTemplate(
    input_variables=["input"], template="given the input, extract the answer as a yes or no with no explanation or additional text. Input: {input}"
)
extractor_chain = LLMChain(llm=chat, prompt=extractor_prompt)

# The overall chain
ss_chain = SimpleSequentialChain(chains=[solver_chain, extractor_chain]) # ,verbose=True

print("Answer with explicit reasoning")
avg_cost = 0
for i in range(10):
  with get_openai_callback() as cb:
    result = ss_chain.invoke(riddle_prompt)
    print("Answer: ", result["output"], "Cost: $", cb.total_cost)
    avg_cost += cb.total_cost

avg_cost = avg_cost/10
print("Average cost: $", avg_cost)

Answer with explicit reasoning
Answer:  No Cost: $ 0.00015795
Answer:  No. Cost: $ 0.00016605
Answer:  No Cost: $ 0.00013245
Answer:  No Cost: $ 0.00015945
Answer:  No Cost: $ 0.00015194999999999998
Answer:  No Cost: $ 0.00015869999999999998
Answer:  No Cost: $ 0.00016545
Answer:  No Cost: $ 0.00015045
Answer:  No Cost: $ 0.00015644999999999998
Answer:  No Cost: $ 0.0001437
Average cost: $ 0.00015426


There are many forms of *CoT* with mixed results. You may consult the following link for more [info](https://www.prompthub.us/blog/chain-of-thought-prompting-guide#automatic-chain-of-thought-prompting). You may try different forms and see how well they work.

---



## 1.3 ReAct Framework

Sometimes, answering a question may require *thinking* and then *acting* by using some tools. That is, the model may need to take several steps and obtain intermediate answers, just like in *CoT*. Some of these steps may require taking actions (i.e., using tools). This framework is called **ReAct (Reasoning-Acting)**.

To create such a ReAct agent, we use the [`create_react_agent`](https://python.langchain.com/api_reference/langchain/agents/langchain.agents.react.agent.create_react_agent.html#langchain.agents.react.agent.create_react_agent) function.



Let's say our agent needs to answer a complex question that may require being broken down into several steps. For example, consider the following questions which may occur in a customer facing chatbot.

In [ ]:
# question = "Given today's date, look at the price of Microsoft stock. If I had bought 1 stock 3 years ago, how much money would I make today by selling?" # Given today's date,


question = "Create a task in my project with name LLMtest. The task's type is bug with name as tomorrow's date and assignee is mentormohannad@gmail.com"
# question = "How many tasks are there with 2024-11-14 as a name in project with name LLMtest"
# question = "How many tasks are there with yesterday's date as a name in project with name LLMtest"
# question = "Who is the assignee of the task with yesterday's date as a name in project with name LLMtest"

It seems to answer such questions, we need ReAct to reason and act using:

1. A tool that is good at calculations.
2. A tool that can search the internet.
3. A tool that can find today's date.
4. A tool that can read and create Jira issues.
  *   For Jira, you simply need to [sign up](https://www.atlassian.com/try/cloud/signup?bundle=jira-software&edition=free&skipBundles=true) first, create a project and [get a token](https://id.atlassian.com/manage-profile/security) for authentication. After that, the [Jira toolkit](https://python.langchain.com/docs/integrations/tools/jira/) can be used.





In [ ]:
!pip install --upgrade --quiet  atlassian-python-api
!pip install -qU langchain-community langchain_openai

In [ ]:
from langchain import hub
from langchain_community.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent, create_tool_calling_agent, tool
from langchain_community.tools.google_finance import GoogleFinanceQueryRun
from langchain import LLMMathChain

### Get the OpenAI API key
from google.colab import userdata
openai_api_key = userdata.get('MyOpenAIKey')

# LLM
chat = ChatOpenAI(openai_api_key=openai_api_key, model="gpt-4o-mini")

In [ ]:

## tool 1: getting today's date. Already defined above.
from datetime import date, timedelta, datetime
@tool
def datetoday(dummy: str) -> str:
    """Returns today's date, use this for any \
    questions that need today's date to be answered. \
    This tool returns a string with today's date.""" #This is the desciption the agent uses to determine whether to use the time tool.
    return "Today is " + str(date.today())

In [ ]:
## tool 2: Calculator.
# We can simply define a tool that uses the LLMChain above
llm = OpenAI(openai_api_key=openai_api_key)
llm_math = LLMMathChain.from_llm(llm)
@tool
def calculator_tool(input_message: str) -> str:
    """Will not be used for any date arithmetics. Takes input message and performs the necessary calculations.
    Returns the result.""" #This is the desciption the agent uses to determine whether to use the time tool.
    return "The answer to " + input_message + " is " + llm_math.invoke(input_message)["answer"]


<ipython-input-18-fe299cf11e23>:3: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(openai_api_key=openai_api_key)


In [ ]:
## tool 3: Internet Search.

from langchain_core.tools import Tool
from langchain_community.utilities import GoogleSerperAPIWrapper
import os

# Setting up the Serper tool
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API')
search = GoogleSerperAPIWrapper()
serper_tool = Tool(
        name="GoogleSerper",
        func=search.run,
        description="Useful for when you need to look up some information on the internet.",
    )


In [ ]:
## toolkit 1
import os
from langchain_community.agent_toolkits.jira.toolkit import JiraToolkit
from langchain_community.utilities.jira import JiraAPIWrapper

os.environ["JIRA_API_TOKEN"] = userdata.get('JIRA_API_TOKEN')
os.environ["JIRA_USERNAME"] = "mentormohannad@gmail.com" #Email used to sign in to JIRA
os.environ["JIRA_INSTANCE_URL"] = "https://mohannadelhamod.atlassian.net/" #URL used to reach your free JIRA instance
os.environ["JIRA_CLOUD"] = "True"

jira = JiraAPIWrapper()
toolkit = JiraToolkit.from_jira_api_wrapper(jira)

# DON'T CHANGE: Some fixes and improvements needed.
for idx, tool in enumerate(toolkit.tools):
    toolkit.tools[idx].name = toolkit.tools[idx].name.replace(" ", "_")
toolkit.tools[2].description = toolkit.tools[2].description + ". Don't set priority for issues. Make sure to specify the project ID." # Future work: could set the assignee to {'accountId':'account_id'}


Now, let's put the tools together

In [ ]:
tools = [
    datetoday,
    calculator_tool,
    serper_tool
] + toolkit.get_tools()

Let's take a look at the different tools available

In [ ]:
for idx, tool in enumerate(tools):
    print(f"Tool {idx}: {tool.name}")
    print("Description:", tool.description)

Tool 0: datetoday
Description: Returns today's date, use this for any     questions that need today's date to be answered.     This tool returns a string with today's date.
Tool 1: calculator_tool
Description: Will not be used for any date arithmetics. Takes input message and performs the necessary calculations.
    Returns the result.
Tool 2: GoogleSerper
Description: Useful for when you need to look up some information on the internet.
Tool 3: JQL_Query
Description: 
    This tool is a wrapper around atlassian-python-api's Jira jql API, useful when you need to search for Jira issues.
    The input to this tool is a JQL query string, and will be passed into atlassian-python-api's Jira `jql` function,
    For example, to find all the issues in project "Test" assigned to the me, you would pass in the following string:
    project = Test AND assignee = currentUser()
    or to find issues with summaries that contain the word "test", you would pass in the following string:
    summary ~ 't

Let's take look a look at the ReAct prompt and how it directs the model to *"think"*...

In [ ]:
prompt = hub.pull("hwchase17/react")
prompt

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

Now, let's see the thinking and acting in motion!

In [ ]:
### You can turn on debugging using this code. You will be able to see the intermediate requests and responses.
import langchain
langchain.debug = False

# The following prompt is designed for ReAct framework.
prompt = hub.pull("hwchase17/react")
agent = create_react_agent(chat, tools, prompt) # Now we have a ReAct agent!

# We may still use create_tool_calling_agent and get good result. But, it is not following the standard ReAct framework per se.
# More on agent types can be found here: https://python.langchain.com/v0.1/docs/modules/agents/agent_types/
# from langchain_core.prompts import ChatPromptTemplate
# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", "You are a helpful assistant."),
#         ("human", "{input}"),
#         ("placeholder", "{agent_scratchpad}"), # To be used by the agent for intermediate operations.
#     ]
# )
# agent = create_tool_calling_agent(chat, tools, prompt)


agent_executor = AgentExecutor(agent=agent, tools=tools, verbose= True, stream_runnable=False)
agent_executor.invoke({"input": question}, handle_parsing_errors=True)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
I need to create a task in the user's project with specific details. First, I need to retrieve the user's projects to find the appropriate project ID. Then I'll get tomorrow's date to set it as the task's name. Finally, I will create the task with the specified details. 

Action: Get_Projects
Action Input: NoneFound 1 projects:
[{'id': '10000', 'key': 'SCRUM', 'name': 'LLMtest', 'type': 'software', 'style': 'next-gen'}]I have retrieved the project ID for the project named "LLMtest." Now, I need to get tomorrow's date, which I will use as the task's name. 

Action: datetoday
Action Input: dummyToday is 2024-11-18

Tomorrow's date is 2024-11-19. I will now create the bug task with the name "2024-11-19," and assign it to the specified email address.

Action: Create_Issue
Action Input: {"summary": "2024-11-19", "description": "", "issuetype": {"name": "Bug"}, "project": {"id": "10000"}, "assignee": {"emailAddress": "mentormohannad@gmail.com"}}{'id': '10014', 'key': 'SCRUM-15', 'self': 'https://mohannadelhamod.atlassian.net/rest/api/2/issue/10014'}I have successfully created the bug task with the name "2024-11-19" in the project "LLMtest," assigned to the user with the email address mentormohannad@gmail.com. 

Final Answer: The task has been created successfully with the name "2024-11-19" in the project "LLMtest."

> Finished chain.


{'input': "Create a task in my project with name LLMtest. The task's type is bug with name as tomorrow's date and assignee is mentormohannad@gmail.com",
 'output': 'The task has been created successfully with the name "2024-11-19" in the project "LLMtest."'}

# 2. Homework

##2.1 Creating and Dating Files

Write an agent that solves the following question correctly:

> Create a file named *\<some filename\>*.txt and contains today's date.

Once it executes, the agent will be able to create the file with the correct name and content. It should create the file under `/content`. It will show up on the left side of your Google notebook under *files*. **(5 Points)**

You will need to find and integrate the right tool from this [list](https://python.langchain.com/docs/integrations/tools/) to be able to create and write a file.

You will also need to use the `datetoday` tool we defined previously to get today's date.

Make sure you allow the agent to use multiple tools.

**Answer**

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

In [ ]:
filename = "hello.txt" # Your code should respect this desired filename

Here write an instruction prompt that effectively execute the desired functionality. Remember that `filename` is a *variable*. **(4 Points)**

In [ ]:
instruction = ### Fill in code here. This prompt will be used in the prompt template.

In [ ]:
chat = ChatOpenAI(openai_api_key=openai_api_key, model="gpt-4o-mini")

# Create the PromptTemplate from the instruction you defined above.
instruction_template =PromptTemplate(
    input_variables= ### Fill in code here
    template=instruction
    )

Define the first tool that gets today's date

In [ ]:
from datetime import date, timedelta, datetime
@tool
def datetoday() -> str:
    """Returns today's date, use this for any \
    questions that need today's date to be answered. \
    This tool takes no argumetns but returns a string with today's date.""" #This is the desciption the agent uses to determine whether to use the time tool.
    return "Today is " + str(date.today())

Instantiate the second tool that is able to create and write the file. **(7 Points)**

In [ ]:
### Fill in code here to get the tool. DO NOT create a custom tool!

Putting it all together.
When you run the code, *make sure it shows the intermedate tool calls*. **(4 Points)**

In [ ]:
tools = ### Fill here: Since both tools should be available, place them into a list.

# Defining the agent
### You may need to make minor changes here.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)
agent = create_tool_calling_agent(chat, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools) # , verbose=True

formatted_instruction = ### Fill code here
print("Agent answer:", agent_executor.invoke({"input": formatted_instruction})["output"])